<a href="https://colab.research.google.com/github/HaitongZhang/CSST-CL-DA/blob/main/melgan_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rishikksh20/melgan.git

Cloning into 'melgan'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 171 (delta 47), reused 44 (delta 44), pack-reused 114 (from 1)
Receiving objects: 100% (171/171), 698.93 KiB | 2.23 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [10]:
# For Full-Band MelGAN
!gdown https://drive.google.com/uc?id=1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID # https://drive.google.com/file/d/1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID/view?usp=sharing

# For Multi-Band MelGAN
!gdown https://drive.google.com/uc?id=1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex # https://drive.google.com/file/d/1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID
From (redirected): https://drive.google.com/uc?id=1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID&confirm=t&uuid=90ef8047-5147-48f8-970b-b3c208230b4f
To: /content/melgan/second_9323377_0930.pt
100% 109M/109M [00:01<00:00, 84.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex
From (redirected): https://drive.google.com/uc?id=1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex&confirm=t&uuid=6d7049d2-0dbd-4335-bcff-f33dc2d45ed7
To: /content/melgan/mb_melgan_901be72_0600.pt
100% 82.9M/82.9M [00:02<00:00, 29.6MB/s]


In [3]:
!pip install -r melgan/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.2 MB/s eta 0:00:00


In [4]:
!pip install unidecode
!pip3 install pesq
!pip3 install pystoi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.3 MB/s eta 0:00:00


In [5]:
%cd melgan/

/content/melgan


In [6]:
import torch
import numpy as np
from model.generator import Generator
from utils.hparams import HParam, load_hparam_str
from utils.pqmf import PQMF

In [12]:
%ls /content/melgan

assets/            dump/                      melgan_infer.ipynb     README.md               utils/
config/            generator.txt              model/                 requirements.txt
datasets/          inference.py               multi-band-melgan.txt  sample/
denoiser.py        LICENSE                    pred_mel.npy           second_9323377_0930.pt
discriminator.txt  mb_melgan_901be72_0600.pt  preprocess.py          trainer.py


In [19]:
text = "To improve the alignment accuracy and thus reduce the information gap."

In [ ]:
# Load Tacotron

In [14]:
# tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2').cuda()
# # preprocessing
# sequence = np.array(tacotron2.text_to_sequence(text, ['english_cleaners']))[None, :]
# sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)

# # run the models
# with torch.no_grad():
#     _, mel, _, _ = tacotron2.infer(sequence)

In [20]:
!pip install numpy scipy librosa unidecode inflect librosa
!apt-get update
!apt-get install -y libsndfile1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,517 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,830 kB]
Fetched 4,475 kB in 2s (2,967 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file '

In [21]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/entrypoints.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untr

Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0-2): 3 x Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (linear_layer): Linear(in_features=80, out_features=256, bias=False)
        )
        (1): LinearNorm(
          (linear_layer): Linear(in_features=256, out_features=256, bias=False)
        )
      )
    )
    (attention_rnn): LSTMCell(768, 1024)
    (attention_layer): Attention(
      (query_layer): LinearNorm(
        (linear_layer): Linear(in_features=1024, out_features=128, bias=False)
      )
      (memory_layer): LinearNorm(
        (linear_layer): Linear(in_fea

In [22]:
text = "To improve the alignment accuracy and thus reduce the information gap."

In [23]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'


In [24]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)

In [25]:
print(mel.shape)

torch.Size([1, 80, 189])


In [ ]:
# MelGAN

In [23]:
print(hp.model.out_channels)

1


In [ ]:
from pesq import pesq
from pystoi import stoi
import torchaudio
from torchaudio.pipelines import SQUIM_OBJECTIVE, SQUIM_SUBJECTIVE
import torchaudio.functional as F

In [72]:
checkpoint = torch.load('/content/melgan/second_9323377_0930.pt')
hp = load_hparam_str(checkpoint['hp_str'])

vocoder_full = Generator(hp.audio.n_mel_channels, hp.model.n_residual_layers,
                        ratios=hp.model.generator_ratio, mult = hp.model.mult,
                        out_band = hp.model.out_channels).cuda()
vocoder_full.load_state_dict(checkpoint['model_g'])
vocoder_full.eval(inference=False)

<ipython-input-72-15efe232e8e4>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/melgan/second_9323377_0930.pt')
/usr/local/lib/python3.10/

In [73]:
MAX_WAV_VALUE = 32768.0
import time
start = time.time()
for i in range(1000):
  with torch.no_grad():
      mel = mel.detach()
      if len(mel.shape) == 2:
          mel = mel.unsqueeze(0)
      mel = mel.cuda()
      audio = vocoder_full.inference(mel)
      # For multi-band inference
      # if hp.model.out_channels > 1:
      #     pqmf = PQMF()
      #     audio = pqmf.synthesis(audio).view(-1)

      audio = audio.squeeze() # collapse all dimension except time axis
      audio = audio[:-(hp.audio.hop_length*10)]
      audio = MAX_WAV_VALUE * audio
      audio = audio.clamp(min=-MAX_WAV_VALUE, max=MAX_WAV_VALUE-1)
      audio_tensor_full = audio
      audio = audio.short()
      audio = audio.cpu().detach().numpy()
end = time.time()
print("fullband melgan takes ", end-start,"s")

fullband melgan takes  15.030830383300781 s


In [75]:
from IPython.display import Audio
Audio(audio, rate = hp.audio.sampling_rate)

In [76]:
WAVEFORM_SPEECH, SAMPLE_RATE_SPEECH = audio_tensor_full, 22050
if SAMPLE_RATE_SPEECH != 16000:
    WAVEFORM_SPEECH = F.resample(WAVEFORM_SPEECH, SAMPLE_RATE_SPEECH, 16000)

print(WAVEFORM_SPEECH.device)
WAVEFORM_SPEECH = WAVEFORM_SPEECH.unsqueeze(0).cpu()

objective_model = SQUIM_OBJECTIVE.get_model()
stoi_hyp, pesq_hyp, si_sdr_hyp = objective_model(WAVEFORM_SPEECH[0:1, :])
print(f"STOI: {stoi_hyp[0]}")
print(f"PESQ: {pesq_hyp[0]}")
print(f"SI-SDR: {si_sdr_hyp[0]}\n")

cuda:0
STOI: 0.9824873805046082
PESQ: 3.1027839183807373
SI-SDR: 23.274002075195312



In [77]:
checkpoint = torch.load('/content/melgan/mb_melgan_901be72_0600.pt')
hp = load_hparam_str(checkpoint['hp_str'])

vocoder = Generator(hp.audio.n_mel_channels, hp.model.n_residual_layers,
                        ratios=hp.model.generator_ratio, mult = hp.model.mult,
                        out_band = hp.model.out_channels).cuda()
vocoder.load_state_dict(checkpoint['model_g'])
vocoder.eval(inference=False)

<ipython-input-77-1cf23cd0ecbf>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/melgan/mb_melgan_901be72_0600.pt')


In [78]:
MAX_WAV_VALUE = 32768.0
import time
start = time.time()
for i in range(1000):
  with torch.no_grad():
      mel = mel.detach()
      if len(mel.shape) == 2:
          mel = mel.unsqueeze(0)
      mel = mel.cuda()
      audio = vocoder.inference(mel)
      # For multi-band inference
      if hp.model.out_channels > 1:
          pqmf = PQMF()
          audio = pqmf.synthesis(audio).view(-1)

      audio = audio.squeeze() # collapse all dimension except time axis
      audio = audio[:-(hp.audio.hop_length*10)]
      audio = MAX_WAV_VALUE * audio
      audio = audio.clamp(min=-MAX_WAV_VALUE, max=MAX_WAV_VALUE-1)
      audio_tensor = audio
      audio = audio.short()
      audio = audio.cpu().detach().numpy()
end = time.time()
print("multiband melgan takes ", end-start,"s")

fullband melgan takes  8.046419858932495 s


In [79]:
from IPython.display import Audio
Audio(audio, rate = hp.audio.sampling_rate)

Automatic estimate the quality of synthesize speech using stoi, pesq, si_sdr

In [80]:
WAVEFORM_SPEECH, SAMPLE_RATE_SPEECH = audio_tensor, 22050
if SAMPLE_RATE_SPEECH != 16000:
    WAVEFORM_SPEECH = F.resample(WAVEFORM_SPEECH, SAMPLE_RATE_SPEECH, 16000)

print(WAVEFORM_SPEECH.device)
WAVEFORM_SPEECH = WAVEFORM_SPEECH.unsqueeze(0).cpu()

objective_model = SQUIM_OBJECTIVE.get_model()
stoi_hyp, pesq_hyp, si_sdr_hyp = objective_model(WAVEFORM_SPEECH[0:1, :])
print(f"STOI: {stoi_hyp[0]}")
print(f"PESQ: {pesq_hyp[0]}")
print(f"SI-SDR: {si_sdr_hyp[0]}\n")

cuda:0
STOI: 0.9755051732063293
PESQ: 3.211214303970337
SI-SDR: 21.973052978515625



for inference speed

i run the inference step for 100o times to get a more stable restimate of inference time.
for the original version, it took 15.030830383300781s;
for the multi-band version, it took 8.046419858932495s;

Thus, there is ~46% inprovement using multi-band to optimize the inference speed in this case.

for full-band/original melgan, the quality metrics are:
STOI: 0.9824873805046082
PESQ: 3.1027839183807373
SI-SDR: 23.274002075195312

for multi-band melgan, the quality metrics are:
STOI: 0.9755051732063293
PESQ: 3.211214303970337
SI-SDR: 21.973052978515625

Note: the higher value, the better the quality.

In summary, multi-band melgan out-performs the original one in terms of PESQ, while the performance of multi-band version get a bit worse on STOI and SI-SNR.


Q4: Given this simple model, do you have any ideas to improve its performance (inference speed, audio quality, …)  if you have a chance to retrain it with a few slight changes from the model's logic and the loss function?. If yes, please implement your suggestions.
*italicized text*



For this question, i think melgan can be improved using extra loss functions, such as Multi-Period Discrimnative loss at the dircrimitive side, and multi-STFT loss at the generative size.

In [ ]:
class MultiPeriodDiscriminator(torch.nn.Module):
    def __init__(self):
        super(MultiPeriodDiscriminator, self).__init__()
        self.discriminators = nn.ModuleList([
            DiscriminatorP(2),
            DiscriminatorP(3),
            DiscriminatorP(5),
            DiscriminatorP(7),
            DiscriminatorP(11),
        ])

    def forward(self, y, y_hat):
        y_d_rs = []
        y_d_gs = []
        fmap_rs = []
        fmap_gs = []
        for i, d in enumerate(self.discriminators):
            y_d_r, fmap_r = d(y)
            y_d_g, fmap_g = d(y_hat)
            y_d_rs.append(y_d_r)
            fmap_rs.append(fmap_r)
            y_d_gs.append(y_d_g)
            fmap_gs.append(fmap_g)

        return y_d_rs, y_d_gs, fmap_rs, fmap_gs

In [99]:
import math
import os
import random
import torch
import torch.utils.data
import numpy as np
from librosa.util import normalize
from scipy.io.wavfile import read
from librosa.filters import mel as librosa_mel_fn

MAX_WAV_VALUE = 32768.0


def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate


def dynamic_range_compression(x, C=1, clip_val=1e-5):
    return np.log(np.clip(x, a_min=clip_val, a_max=None) * C)


def dynamic_range_decompression(x, C=1):
    return np.exp(x) / C


def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)


def dynamic_range_decompression_torch(x, C=1):
    return torch.exp(x) / C


def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output


def spectral_de_normalize_torch(magnitudes):
    output = dynamic_range_decompression_torch(magnitudes)
    return output


mel_basis = {}
hann_window = {}


def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        # mel = librosa_mel_fn(sampling_rate, n_fft, num_mels, fmin, fmax). # maybe version problem
        mel = librosa_mel_fn(sr=sampling_rate, n_fft=n_fft, n_mels=num_mels, fmin=fmin, fmax=fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True, return_complex=False)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec


def get_dataset_filelist(a):
    with open(a.input_training_file, 'r', encoding='utf-8') as fi:
        training_files = [os.path.join(a.input_wavs_dir, x.split('|')[0] + '.wav')
                          for x in fi.read().split('\n') if len(x) > 0]

    with open(a.input_validation_file, 'r', encoding='utf-8') as fi:
        validation_files = [os.path.join(a.input_wavs_dir, x.split('|')[0] + '.wav')
                            for x in fi.read().split('\n') if len(x) > 0]
    return training_files, validation_files


class MelDataset(torch.utils.data.Dataset):
    def __init__(self, training_files, segment_size, n_fft, num_mels,
                 hop_size, win_size, sampling_rate,  fmin, fmax, split=True, shuffle=True, n_cache_reuse=1,
                 device=None, fmax_loss=None, fine_tuning=False, base_mels_path=None):
        self.audio_files = training_files
        random.seed(1234)
        if shuffle:
            random.shuffle(self.audio_files)
        self.segment_size = segment_size
        self.sampling_rate = sampling_rate
        self.split = split
        self.n_fft = n_fft
        self.num_mels = num_mels
        self.hop_size = hop_size
        self.win_size = win_size
        self.fmin = fmin
        self.fmax = fmax
        self.fmax_loss = fmax_loss
        self.cached_wav = None
        self.n_cache_reuse = n_cache_reuse
        self._cache_ref_count = 0
        self.device = device
        self.fine_tuning = fine_tuning
        self.base_mels_path = base_mels_path

    def __getitem__(self, index):
        filename = self.audio_files[index]
        if self._cache_ref_count == 0:
            audio, sampling_rate = load_wav(filename)
            audio = audio / MAX_WAV_VALUE
            if not self.fine_tuning:
                audio = normalize(audio) * 0.95
            self.cached_wav = audio
            if sampling_rate != self.sampling_rate:
                raise ValueError("{} SR doesn't match target {} SR".format(
                    sampling_rate, self.sampling_rate))
            self._cache_ref_count = self.n_cache_reuse
        else:
            audio = self.cached_wav
            self._cache_ref_count -= 1

        audio = torch.FloatTensor(audio)
        audio = audio.unsqueeze(0)

        if not self.fine_tuning:
            if self.split:
                if audio.size(1) >= self.segment_size:
                    max_audio_start = audio.size(1) - self.segment_size
                    audio_start = random.randint(0, max_audio_start)
                    audio = audio[:, audio_start:audio_start+self.segment_size]
                else:
                    audio = torch.nn.functional.pad(audio, (0, self.segment_size - audio.size(1)), 'constant')

            mel = mel_spectrogram(audio, self.n_fft, self.num_mels,
                                  self.sampling_rate, self.hop_size, self.win_size, self.fmin, self.fmax,
                                  center=False)
        else:
            mel = np.load(
                os.path.join(self.base_mels_path, os.path.splitext(os.path.split(filename)[-1])[0] + '.npy'))
            mel = torch.from_numpy(mel)

            if len(mel.shape) < 3:
                mel = mel.unsqueeze(0)

            if self.split:
                frames_per_seg = math.ceil(self.segment_size / self.hop_size)

                if audio.size(1) >= self.segment_size:
                    mel_start = random.randint(0, mel.size(2) - frames_per_seg - 1)
                    mel = mel[:, :, mel_start:mel_start + frames_per_seg]
                    audio = audio[:, mel_start * self.hop_size:(mel_start + frames_per_seg) * self.hop_size]
                else:
                    mel = torch.nn.functional.pad(mel, (0, frames_per_seg - mel.size(2)), 'constant')
                    audio = torch.nn.functional.pad(audio, (0, self.segment_size - audio.size(1)), 'constant')

        mel_loss = mel_spectrogram(audio, self.n_fft, self.num_mels,
                                   self.sampling_rate, self.hop_size, self.win_size, self.fmin, self.fmax_loss,
                                   center=False)

        return (mel.squeeze(), audio.squeeze(0), filename, mel_loss.squeeze())

    def __len__(self):
        return len(self.audio_files)



In [104]:
import torch.nn.functional as F

In [107]:


def multi_stft_loss(pred, gt, sampling_rate, fmin, fmax, num_mels, n_ffts,  hop_sizes, win_sizes, loss_weight):
  loss = 0.

  for n_fft, hop_size, win_size in zip(n_ffts, hop_sizes, win_sizes):
    pred_mel = mel_spectrogram(pred, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax)
    gt_mel = mel_spectrogram(gt, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax)

    loss +=  F.l1_loss(gt_mel, pred_mel)

  loss *= loss_weight
  return loss



In [108]:
# just take these two as an example
pred = audio_tensor_full.unsqueeze(0)
gt = audio_tensor.unsqueeze(0)
fmin = 0
fmax = 22050 // 2
n_ffts = [1024, 2048]
hop_sizes = [128, 256] # 22050
win_sizes = [128*4, 256*4]
num_mels = 80
loss_weight = 1.0 # hyper-parameter to be tuned
loss = multi_stft_loss(pred, gt, 22050, fmin, fmax, num_mels, n_ffts, hop_sizes, win_sizes, loss_weight)



[1024, 2048] [128, 256] [512, 1024]
min value is  tensor(-11042.0732, device='cuda:0')
max value is  tensor(15692.0557, device='cuda:0')
min value is  tensor(-10836.7217, device='cuda:0')
max value is  tensor(17727.4785, device='cuda:0')
min value is  tensor(-11042.0732, device='cuda:0')
max value is  tensor(15692.0557, device='cuda:0')
min value is  tensor(-10836.7217, device='cuda:0')
max value is  tensor(17727.4785, device='cuda:0')


In [110]:
print(loss.item())

0.9865882992744446
